Keras는 딥러닝 모델을 개발하고 평가할 수있는 강력하고 사용하기 쉬운 Python 라이브러리다. 효율적인 수치 계산 라이브러리인 Theano와 TensorFlow를 백엔드에 두고, 단 줄의 코드로 신경망 모델을 구축하고 학습할 수 있다.

여기에서는 Keras를 사용하여 간단한 신경망 모델을 만드는 방법을 살펴보고자 한다. 구체적으로, 다음과 같은 사항에 대해 이야기할 것이다.

- Keras와 함께 사용할 준비가 된 CSV 데이터 세트를 로드하는 방법.
- Keras에서 Multilayer perceptron 모델을 정의하고 컴파일하는 방법.

# 개요

본 튜토리얼에서는 다음과 같은 순서대로 진행할 것이다.

1. 데이터 불러오기
2. 모델 정의
3. 모델 컴파일
4. 모델 적합
5. 모델 평가

# Pima Indians Onset of Diabetes Dataset

이 실습을 위해, Pima Indians의 당뇨병 dataset을 사용할 것이다. 이 dataset은 UCI Machine Learning repository에서 구할 수 있다.

http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data 

Pima indians에 속한 환자의 의료 기록 데이터와, 5년 이내에 당뇨병 진단을 받았는지 여부를 설명하기 위한 데이터이다. 이 데이터가 어려가지 목적 하에서 활용될 수 있겠지만, 여기에서는 당뇨병 진단을 받은 경우를 1, 아닌 경우를 0으로 두고 binary classification을 할 것이다.

이 데이터에서 제공하는 변수는 다음과 같다. 모든 데이터는 수치 데이터이다.

1. Number of times pregnant
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
3. Diastolic blood pressure (mm Hg)
4. Triceps skin fold thickness (mm)
5. 2-Hour serum insulin (mu U/ml).
6. Body mass index.
7. Diabetes pedigree function.
8. Age (years).
9. Class, onset of diabetes within five years.

총 768개의 인스턴스가 존재하며, 처음 5번째 행은 다음과 같은 모습이다.

6,148,72,35,0,33.6,0.627,50,1

1,85,66,29,0,26.6,0.351,31,0

8,183,64,0,0,23.3,0.672,32,1

1,89,66,23,94,28.1,0.167,21,0

0,137,40,35,168,43.1,2.288,33,1

모든 변수가 수치형 데이터라는 점에서, 기본적으로 수치형 데이터의 입출력을 기대하는 신경망에 직접 적합하기에 매우 용이할 것이다.

# 데이터 불러오기

Stochastic process를 사용하는 기계학습 알고리즘을 사용할 때마다, 고정 시드 값으로 난수 생성 프로그램을 초기화하는 것이 좋다. 이렇게 하면, 동일한 코드를 반복해서 실행할 때마다 언제나 동일한 결과를 얻을 수 있다. 특히 이 방법은, 다른 사람들에게 결과를 보여주거나, 다른 알고리즘과 성능을 비교하거나, 코드를 디버깅할 때 유용하다.

파이썬에서는 numpy의 random.seed 메서드를 이용한다.

In [13]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

# 고정 시드 값으로 초기화
numpy.random.seed(123)

이제 Pima Indians dataset을 불러오자. CSV 파일은 numpy 라이브러리의 loadtxt() 함수를 사용하여 직접 불러올 수 있다. Pima Indians dataset에는 8개의 입력 변수와 1개의 출력 변수(마지막 열)가 있다. 일단 dataset을 불러오면, 입력변수(X)와 출력변수(Y)를 나눌 수 있다.

In [8]:
# Pima Indians Diabetes 데이터 불러오기
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# 입력 변수와 출력 변수를 구분하기
X = dataset[:,0:8]
Y = dataset[:,8]

# 모델 정의

Keras의 모델은 일련의 레이어로 정의된다. 네트워크 토폴로지에 만족할 때까지 순차 모델을 만들고 레이어를 하나씩 추가한다. 첫 번째로 확인해야할 것은, 입력 레이어에 올바른 입력 수가 있는지 확인하는 것이다. 이것은 input_dim 인수로 첫 번째 레이어를 만들 때 지정할 수 있다. (그에 앞서, 해당 변수의 dimension을 확인하기 위해서는 shape() 메서드를 사용할 수 있다.)

적절한 레이어의 수와 유형을 어떻게 알 수 있을까? 이것은 매우 어려운 질문이다. 보통 우리는 이 레이어 수와 유형을 계속 변경하고 시행 착오를 겪으면서 최상의 네트워크 구조를 발견해내는 식이다. 일반적으로 문제의 구조를 파악하기 위해서는 충분한 수의 네트워크가 필요하다.

이 예제에서는 3 개의 레이어가 있는 완전히 연결된(fully connected) 네트워크 구조를 사용하기로 한다.

완전히 연결된(fully connected) 레이어는 Dense 클래스를 사용하여 정의된다. 레이어의 뉴런 수(노드 수)를 첫 번째 인수로 지정하고, 두 번째 인수로서 초기화 메서드를 init으로 지정하고, activation 인수를 사용하여 활성화 함수를 지정할 수 있다. 이 경우 Keras의 기본 가중치 초기화는 Uniform[0, 0.5]에서 생성 된 작은 난수로 네트워크 가중치를 초기화한다. 이와 다른 전통적인 대안으로는 Gaussian 분포로부터 생성하기도 한다.

처음 두 레이어에는 RELU 활성화 함수를 사용하고 출력 레이어에는 Sigmoid 활성화 함수를 사용하였다. 이전에는 Sigmoid 혹은 Tan 활성화 함수가 모든 레이어에서 선호되어왔지만, 오늘날은 조금 다양한 함수를 활용하는 추세이고 각 활성화 함수마다 장단점을 인지하며 사용하려고도 하는 편이다.

In [9]:
X.shape

(768, 8)

In [10]:
# 모델 정의
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

지금까지 설명한 모델은, 아래 그림으로 표현할 수 있다.

<img src="004_001.jpg">

# 모델 컴파일

모델이 정의되었으므로 이제 컴파일 할 수 있다. 모델을 컴파일할 때는 백엔드에서 Theano 또는 TensorFlow와 같은 효율적인 수치 라이브러리를 사용하게 된다. 백엔드는 학습을 위해 네트워크를 표현하고 하드웨어에서 예측을 수행하는 가장 좋은 방법을 자동으로 선택한다. 컴파일 할 때, 네트워크를 학습에 필요한 몇 가지 추가 속성을 지정해야한다.

우리는 가중치 셋을 평가하는 데 사용할 손실 함수, 네트워크의 다른 가중치를 탐색하는 데 사용되는 최적화 알고리즘 및 학습 중에 수집하고 보고하고자하는 선택적인 측정 기준을 지정해야 한다. 이 경우 여기에서는 logarithmic loss를 사용할 것인데, 이진 분류 문제는 Keras에서 binary_crossentropy로 정의한다. 또 여기에서는 효율적인 그라디언트 디센트 알고리즘 중 하나인 Adam을 사용할 것이다. 마지막으로 분류 문제이므로 분류 정확도를 확인하기 위해 accuracy를 수집하여 보고하기로 한다.

In [11]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모형 적합 (fit model)

우리는 모델을 정의하고 효율적인 계산을 위한 준비를 끝냈다. 이제는 데이터에로부터 모델을 fitting할 차례이다. fit() 함수를 호출하여 로드 된 데이터에 모델을 적용하거나 조정할 수 있습니다.

학습 과정은 epochs라는 '데이터 세트를 통해 고정 된 반복 횟수' 동안 실행되며, epochs 인수를 사용하여 얼마나 반복할 것인지를 지정해야 한다. 네트워크에서 batch_size 인수를 조정하여, 데이터를 한 번에 처리하지 않고 일부분씩 나누어 처리할 수도 있다.

In [14]:
# 모형 적합
model.fit(X, Y, epochs=150, batch_size=10)

Epoch 1/150
768/768 [==============================] - 3s - loss: 3.7357 - acc: 0.5990     
Epoch 2/150
768/768 [==============================] - 0s - loss: 0.9400 - acc: 0.5911     
Epoch 3/150
768/768 [==============================] - 0s - loss: 0.7509 - acc: 0.6393     
Epoch 4/150
768/768 [==============================] - 0s - loss: 0.7113 - acc: 0.6589     
Epoch 5/150
768/768 [==============================] - 0s - loss: 0.6799 - acc: 0.6784     
Epoch 6/150
768/768 [==============================] - 0s - loss: 0.6493 - acc: 0.6823     
Epoch 7/150
768/768 [==============================] - 0s - loss: 0.6486 - acc: 0.6706     
Epoch 8/150
768/768 [==============================] - 0s - loss: 0.6359 - acc: 0.6875     
Epoch 9/150
768/768 [==============================] - 0s - loss: 0.6230 - acc: 0.6888     
Epoch 10/150
768/768 [==============================] - 0s - loss: 0.6285 - acc: 0.6771     
Epoch 11/150
768/768 [==============================] - 0s - loss: 0.6448 - acc

여기에서의 작업이, CPU 또는 GPU에서 이루어진다.

# 모델 평가

우리는 전체 dataset으로부터 신경망을 학습시켰으므로, 이제 동일한 dataset을 이용하여 네트워크의 성능을 평가할 수 있다.

(다만, 이렇게 하면 dataset이 얼마나 잘 모델링되었는지는 알 수 있지만, 알고리즘이 새 데이터에 대해서는 얼마나 잘 수행하는지 알 수 없다. 이를 위해 모델의 training 및 test를 위한 dataset을 따로 분리할 수 있다. 이에 대해서는 나중에 다룰 것이다.)

모델의 evaluation() 함수를 사용하여 training dataset에서 모델을 평가하고, 모델을 학습하는 데 사용된 것과 동일한 입력 및 출력을 전달할 수 있다.

In [12]:
# 모델 평가
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/768 [>.............................] - ETA: 46s
acc: 64.97%


# 종합적으로 살펴보기.

Keras에서 첫 번째 신경망 모델을 쉽게 만드는 방법을 살펴보았다. 지금까지 진행한 것을 종합하여 살펴보면 다음과 같다.

In [14]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

# 고정 시드 값으로 초기화
numpy.random.seed(123)



# Pima Indians Diabetes 데이터 불러오기
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# 입력 변수와 출력 변수를 구분하기
X = dataset[:,0:8]
Y = dataset[:,8]



# 모델 정의
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))



# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



# 모형 적합
model.fit(X, Y, epochs=150, batch_size=10)



# 모델 평가
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/150
768/768 [==============================] - 0s - loss: 3.6741 - acc: 0.6055     
Epoch 2/150
768/768 [==============================] - 0s - loss: 3.2995 - acc: 0.6029     
Epoch 3/150
768/768 [==============================] - 0s - loss: 2.9690 - acc: 0.6042     
Epoch 4/150
768/768 [==============================] - 0s - loss: 1.9584 - acc: 0.5833     
Epoch 5/150
768/768 [==============================] - 0s - loss: 1.3228 - acc: 0.6302     
Epoch 6/150
768/768 [==============================] - 0s - loss: 1.1879 - acc: 0.6341     
Epoch 7/150
768/768 [==============================] - 0s - loss: 1.0442 - acc: 0.6302     
Epoch 8/150
768/768 [==============================] - 0s - loss: 0.9332 - acc: 0.6445     
Epoch 9/150
768/768 [==============================] - 0s - loss: 0.8814 - acc: 0.6289     
Epoch 10/150
768/768 [==============================] - 0s - loss: 0.7863 - acc: 0.6510     
Epoch 11/150
768/768 [==============================] - 0s - loss: 0.7357 - acc